<a href="https://colab.research.google.com/github/kasier48/DeepLearning/blob/main/Pratice_Week_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

ds = load_dataset("glue", "mnli")
train_dataset = ds['train']
print(f"Train 데이터 확인: {train_dataset[0]}")

label_info = train_dataset.features['label']
print(f"라벨의 종류: {label_info.names}")

num_labels = len(label_info.names)

dataset_length = 10000
train_dataset = train_dataset.select(range(dataset_length))

validation_matched_dataset = ds['validation_matched']

test_dataset = ds['test_matched']

def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])

    # [MYCODE] 전제와 가설 문장을 합체
    texts.append(row['premise'] + " " + row['hypothesis'])

  encoding = tokenizer(texts, padding=True, truncation=False, max_length=max_len, return_tensors="pt")
  labels = torch.LongTensor(labels)

  return encoding.input_ids, labels, encoding.attention_mask

train_loader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn
)
validation_loader = DataLoader(
    validation_matched_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset, batch_size=64, shuffle=False, collate_fn=collate_fn
)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Train 데이터 확인: {'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'label': 1, 'idx': 0}
라벨의 종류: ['entailment', 'neutral', 'contradiction']


In [ ]:
def train_model(model, optimizer, n_epochs):
  loss_list = []
  train_acc_list = []
  validation_acc_list = []

  for epoch in range(n_epochs):
    total_loss = 0.
    model.train()

    for data in train_loader:
      model.zero_grad()
      inputs, labels, attention_mask = data
      inputs, labels, attention_mask = inputs.to(device), labels.to(device), attention_mask.to(device)

      preds = model(inputs, attention_mask)
      loss = loss_fn(preds, labels)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()

      loss_list.append(total_loss)

    print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

    with torch.no_grad():
      model.eval()
      train_acc = accuracy(model, train_loader)
      train_acc_list.append(train_acc)

      validation_acc = accuracy(model, validation_loader)
      validation_acc_list.append(validation_acc)
      print(f"=========> Train acc: {train_acc:.3f} | Validation acc: {validation_acc:.3f}")


  return (loss_list, train_acc_list, validation_acc_list)

def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels, attention_mask = data
    inputs, labels, attention_mask = inputs.to(device), labels.to(device), attention_mask.to(device)

    preds = model(inputs, attention_mask)
    preds = torch.argmax(preds, dim=-1)

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

from torch import nn

class PreTraiendTextClassifier(nn.Module):
  def __init__(self, num_labels, dropout_rate):
    super(PreTraiendTextClassifier, self).__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.dropout = nn.Dropout(dropout_rate)
    self.classifier = nn.Linear(768, num_labels)

  def forward(self, x, attention_mask):
    x = self.encoder(x, attention_mask)['last_hidden_state']
    x = self.dropout(x[:, 0])
    x = self.classifier(x)
    return x

from torch.optim import Adam
from torch.optim import AdamW
import numpy as np

fine_tunning_model = PreTraiendTextClassifier(num_labels, dropout_rate=0.1)

device = torch.device('cpu')
lr = 1e-5
fine_tunning_model = fine_tunning_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(fine_tunning_model.parameters(), lr=lr, weight_decay=0.01)
# optimizer = Adam(fine_tunning_model.parameters(), lr=lr)
n_epochs = 1

fine_tunning_result = train_model(fine_tunning_model, optimizer, n_epochs)

# =============================================================================================>

from transformers import DistilBertModel, DistilBertConfig

# [MYCODE] Non-traiend 되지 않은 모델 정의
class NonTrainedTextClassifier(nn.Module):
    def __init__(self, num_labels, config, dropout_rate):
        super(NonTrainedTextClassifier, self).__init__()
        self.encoder = DistilBertModel(config)  # 사전 학습되지 않은 DistilBERT
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, x, attention_mask):
        x = self.encoder(x, attention_mask)['last_hidden_state']
        x = self.dropout(x[:, 0])
        x = self.classifier(x)
        return x

non_trained_config = DistilBertConfig()
non_traiend_text_classifier = NonTrainedTextClassifier(num_labels, non_trained_config, dropout_rate=0.1)
non_traiend_text_classifier = non_traiend_text_classifier.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(fine_tunning_model.parameters(), lr=lr, weight_decay=0.01)

non_trained_result = train_model(non_traiend_text_classifier, optimizer, n_epochs)

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt

def plot_acc(first_accs, second_accs, label1='train', label2='test'):
  x = np.arange(len(first_accs))

  plt.plot(x, first_accs, label=label1)
  plt.plot(x, second_accs, label=label2)
  plt.legend()
  plt.show()

fine_tunning_train_acc_list = fine_tunning_result[1]
fine_tunning_validation_acc_list = fine_tunning_result[2]
plot_acc(fine_tunning_train_acc_list, fine_tunning_validation_acc_list, label1="Fine-Tunning-Acc", label2="Validation-Acc")

non_trained_train_acc_list = non_trained_result[1]
non_traiend_validation_acc_list = non_trained_result[2]
plot_acc(non_trained_train_acc_list, non_traiend_validation_acc_list, label1="Non-Train-Acc", label2="Validation-Acc")

fine_tunning_loss_list = fine_tunning_result[0]
non_trained_loss_list = non_trained_result[0]
plot_acc(fine_tunning_loss_list, non_trained_loss_list, label1="Fine-Tunning-Loss", label2="Non-Train-Loss")
